In [38]:
import pandas as pd
from pandasql import PandaSQL
import warnings 
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

In [2]:
productCatalogDf = pd.read_csv('../data/product_catalog_cleaned.csv')

In [3]:
productCatalogDf.head(2)

,Manufacturernumber,Articlenumber,EAN,Technical details,Picture normal reduced,Depth m,Width m,Length m,Weight kg,Delivery time days,...,Language,Manufacturername,Short_description_quality,Short_description_2_quality,Manufacturer_product_count,Product_length_category,Has_image,Volume_m3,Has_long_description,Description_length
0,0 601 6B4 000,06016B4000,NaN,§Titel§Akku-Tauchsäge BITURBO GKT 18V-52 GC Pr...,'https://www.nexmart.com/media/catalog/ampshar...,0.254,0.36,0.444,4.032,NaN,...,de,BOSCH,good,good,140,Medium,1,0.040599,1,1640
1,0 601 6B4 000,06016B4000,NaN,§Titel§Akku-Tauchsäge BITURBO GKT 18V-52 GC Pr...,'https://www.nexmart.com/media/catalog/ampshar...,0.254,0.36,0.444,4.032,NaN,...,en,BOSCH,good,bad,140,Medium,1,0.040599,0,0


In [4]:
productCatalogDf.tail(2)

,Manufacturernumber,Articlenumber,EAN,Technical details,Picture normal reduced,Depth m,Width m,Length m,Weight kg,Delivery time days,...,Language,Manufacturername,Short_description_quality,Short_description_2_quality,Manufacturer_product_count,Product_length_category,Has_image,Volume_m3,Has_long_description,Description_length
306,RALB2EU,RALB2EU,4.012079e+12,§Nennspannung§18§V|§Kapazität§5§Ah|§Ausführung...,'https://www.nexmart.com/media/catalog/ampshar...,NaN,NaN,NaN,0.64,NaN,...,de,BOSCH,good,bad,140,NaN,1,1.0,1,110
307,RALB2US,RALB2US,4.012079e+12,§Nennspannung§18§V|§Kapazität§5§Ah|§Ausführung...,'https://www.nexmart.com/media/catalog/ampshar...,NaN,NaN,NaN,0.64,NaN,...,de,BOSCH,good,bad,140,NaN,1,1.0,1,40


Which manufacturers have the biggest improvement potential in their data quality in absolute and relative numbers?

In [52]:
query = """
SELECT
    Manufacturername,
    COUNT(*) AS total_products,

    -- Count of bad values in selected critical fields
    SUM(CASE WHEN Short_description_quality = 'bad' THEN 1 ELSE 0 END) AS bad_short_desc,
    SUM(CASE WHEN Short_description_2_quality = 'bad' THEN 1 ELSE 0 END) AS bad_short_desc_2,
    SUM(CASE WHEN Has_long_description = 0 THEN 1 ELSE 0 END) AS missing_long_desc,
    SUM(CASE WHEN Has_image = 0 THEN 1 ELSE 0 END) AS missing_image,
    SUM(CASE WHEN EAN IS NULL OR TRIM(EAN) = '' THEN 1 ELSE 0 END) AS missing_ean,
    
    SUM(CASE WHEN [Picture normal reduced] IS NULL OR TRIM([Picture normal reduced]) = '' THEN 1 ELSE 0 END) AS missing_picture_url,
    SUM(CASE WHEN Volume_m3 IS NULL OR Volume_m3 = 0 THEN 1 ELSE 0 END) AS missing_weight,
    SUM(CASE WHEN [Depth m] IS NULL OR [Depth m] = 0 THEN 1 ELSE 0 END) AS missing_depth,
    SUM(CASE WHEN [Width m] IS NULL OR [Width m] = 0 THEN 1 ELSE 0 END) AS missing_width,
    SUM(CASE WHEN [Length m] IS NULL OR [Length m] = 0 THEN 1 ELSE 0 END) AS missing_length,

    -- Total bad fields
    (
        SUM(CASE WHEN Short_description_quality = 'bad' THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Short_description_2_quality = 'bad' THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Has_long_description = 0 THEN 1 ELSE 0 END)+
        SUM(CASE WHEN Has_image = 0 THEN 1 ELSE 0 END) +
        SUM(CASE WHEN EAN IS NULL OR TRIM(EAN) = '' THEN 1 ELSE 0 END) +
        
        SUM(CASE WHEN [Picture normal reduced] IS NULL OR TRIM([Picture normal reduced]) = '' THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Volume_m3 IS NULL OR Volume_m3 = 0 THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Depth m] IS NULL OR [Depth m] = 0 THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Width m] IS NULL OR [Width m] = 0 THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Length m] IS NULL OR [Length m] = 0 THEN 1 ELSE 0 END)
    ) AS total_bad_fields

FROM productCatalogDf
GROUP BY Manufacturername
ORDER BY total_bad_fields DESC;
"""
pandasql_instance = PandaSQL()
result = pandasql_instance(query, locals())
result

,Manufacturername,total_products,bad_short_desc,bad_short_desc_2,missing_long_desc,missing_image,missing_ean,missing_picture_url,missing_weight,missing_depth,missing_width,missing_length,total_bad_fields
0,GUSTAV KLAUKE GMBH,40,0,40,0,4,0,4,0,40,40,40,168
1,FEIN,100,0,100,0,0,0,0,3,3,3,3,112
2,BOSCH,140,4,8,28,0,26,0,0,4,4,4,78
3,ROTHENBERGER,23,0,23,0,0,0,0,0,0,0,0,23
4,FISCHER,5,0,0,0,0,0,0,0,0,0,0,0


In [60]:
query = """
SELECT
    Manufacturername,
    COUNT(*) AS total_products,

    -- Count of bad values in selected critical fields
    SUM(CASE WHEN Short_description_quality = 'bad' THEN 1 ELSE 0 END) AS bad_short_desc,
    SUM(CASE WHEN Short_description_2_quality = 'bad' THEN 1 ELSE 0 END) AS bad_short_desc_2,
    SUM(CASE WHEN Has_long_description = 0 THEN 1 ELSE 0 END) AS missing_long_desc,
    SUM(CASE WHEN Has_image = 0 THEN 1 ELSE 0 END) AS missing_image,
    SUM(CASE WHEN EAN IS NULL OR TRIM(EAN) = '' THEN 1 ELSE 0 END) AS missing_ean,
    SUM(CASE WHEN [Picture normal reduced] IS NULL OR TRIM([Picture normal reduced]) = '' THEN 1 ELSE 0 END) AS missing_picture_url,
    SUM(CASE WHEN Volume_m3 IS NULL OR Volume_m3 = 0 THEN 1 ELSE 0 END) AS missing_weight,
    SUM(CASE WHEN [Depth m] IS NULL OR [Depth m] = 0 THEN 1 ELSE 0 END) AS missing_depth,
    SUM(CASE WHEN [Width m] IS NULL OR [Width m] = 0 THEN 1 ELSE 0 END) AS missing_width,
    SUM(CASE WHEN [Length m] IS NULL OR [Length m] = 0 THEN 1 ELSE 0 END) AS missing_length,
   

    -- Total bad fields
    (
        SUM(CASE WHEN Short_description_quality = 'bad' THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Short_description_2_quality = 'bad' THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Has_long_description = 0 THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Has_image = 0 THEN 1 ELSE 0 END) +
        SUM(CASE WHEN EAN IS NULL OR TRIM(EAN) = '' THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Picture normal reduced] IS NULL OR TRIM([Picture normal reduced]) = '' THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Volume_m3 IS NULL OR Volume_m3 = 0 THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Depth m] IS NULL OR [Depth m] = 0 THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Width m] IS NULL OR [Width m] = 0 THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Length m] IS NULL OR [Length m] = 0 THEN 1 ELSE 0 END)
        
    ) AS total_bad_fields,

    -- Relative bad field percentage 
    ROUND(
        (
            (
                SUM(CASE WHEN Short_description_quality = 'bad' THEN 1 ELSE 0 END) +
                SUM(CASE WHEN Short_description_2_quality = 'bad' THEN 1 ELSE 0 END) +
                SUM(CASE WHEN Has_long_description = 0 THEN 1 ELSE 0 END) +
                SUM(CASE WHEN Has_image = 0 THEN 1 ELSE 0 END) +
                SUM(CASE WHEN EAN IS NULL OR TRIM(EAN) = '' THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Picture normal reduced] IS NULL OR TRIM([Picture normal reduced]) = '' THEN 1 ELSE 0 END) +
                SUM(CASE WHEN Volume_m3 IS NULL OR Volume_m3 = 0 THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Depth m] IS NULL OR [Depth m] = 0 THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Width m] IS NULL OR [Width m] = 0 THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Length m] IS NULL OR [Length m] = 0 THEN 1 ELSE 0 END)
                
                
            ) * 1.0
            /
            (COUNT(*) * 10)
        ) * 100, 2
    ) AS pct_bad_fields

FROM productCatalogDf
GROUP BY Manufacturername
ORDER BY pct_bad_fields DESC;
"""
pandasql_instance = PandaSQL()
result = pandasql_instance(query, locals())
result

,Manufacturername,total_products,bad_short_desc,bad_short_desc_2,missing_long_desc,missing_image,missing_ean,missing_picture_url,missing_weight,missing_depth,missing_width,missing_length,total_bad_fields,pct_bad_fields
0,GUSTAV KLAUKE GMBH,40,0,40,0,4,0,4,0,40,40,40,168,42.00
1,FEIN,100,0,100,0,0,0,0,3,3,3,3,112,11.20
2,ROTHENBERGER,23,0,23,0,0,0,0,0,0,0,0,23,10.00
3,BOSCH,140,4,8,28,0,26,0,0,4,4,4,78,5.57
4,FISCHER,5,0,0,0,0,0,0,0,0,0,0,0,0.00


What product variable/column (description or property) usually contains data of good quality per manufacturer? And what is the % of good quality records per variable/column and manufacturer?

In [91]:
query = """
SELECT
    Manufacturername,

    -- % of good Short description
    ROUND(SUM(CASE WHEN [Short description] IS NOT NULL AND TRIM([Short description]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_good_short_description,

    -- % of good Short description 2
    ROUND(SUM(CASE WHEN [Short description 2] IS NOT NULL AND TRIM([Short description 2]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_good_short_description_2,

    -- % of good Long description
    ROUND(SUM(CASE WHEN [Long description] IS NOT NULL AND TRIM([Long description]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_good_long_description,

    -- % of good Technical details
    ROUND(SUM(CASE WHEN [Technical details] IS NOT NULL AND TRIM([Technical details]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_good_technical_details,

    -- % of good EAN
    ROUND(SUM(CASE WHEN EAN IS NOT NULL AND TRIM(EAN) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_good_ean,

    -- % of products with image (binary field)
    ROUND(SUM(CASE WHEN Has_image = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_has_image,

    -- % with valid image URL
    ROUND(SUM(CASE WHEN [Picture normal reduced] IS NOT NULL AND TRIM([Picture normal reduced]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_good_picture_url,

    -- % of good weight
    ROUND(SUM(CASE WHEN [Weight kg] IS NOT NULL AND [Weight kg] > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_good_weight,

    -- % with full non-zero dimensions
    ROUND(SUM(CASE WHEN [Length m] IS NOT NULL AND [Width m] IS NOT NULL AND [Depth m] IS NOT NULL
                  AND [Length m] > 0 AND [Width m] > 0 AND [Depth m] > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_good_dimensions

FROM productCatalogDf
GROUP BY  Articlenumber
ORDER BY Manufacturername;

"""
pandasql_instance = PandaSQL()
result = pandasql_instance(query, locals())
result

PandaSQLException: (sqlite3.OperationalError) near "Articlenumber": syntax error
[SQL: 
SELECT
    Manufacturername,

    -- % of good Short description
    ROUND(SUM(CASE WHEN [Short description] IS NOT NULL AND TRIM([Short description]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_good_short_description,

    -- % of good Short description 2
    ROUND(SUM(CASE WHEN [Short description 2] IS NOT NULL AND TRIM([Short description 2]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_good_short_description_2,

    -- % of good Long description
    ROUND(SUM(CASE WHEN [Long description] IS NOT NULL AND TRIM([Long description]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_good_long_description,

    -- % of good Technical details
    ROUND(SUM(CASE WHEN [Technical details] IS NOT NULL AND TRIM([Technical details]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_good_technical_details,

    -- % of good EAN
 

Which manufacturers have the most missing or low-quality data per column?
This version gives a row per manufacturer and column-wise missing value counts and percentages.

In [76]:
query = """
SELECT
    Manufacturername,
    COUNT(*) AS total_products,

    -- Missing Short Description
    SUM(CASE WHEN [Short description] IS NULL OR TRIM([Short description]) IN ('', 'N/A') THEN 1 ELSE 0 END) AS missing_short_description,
    ROUND(SUM(CASE WHEN [Short description] IS NULL OR TRIM([Short description]) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_short_description,

    -- Missing Long Description
    SUM(CASE WHEN [Long description] IS NULL OR TRIM([Long description]) IN ('', 'N/A') THEN 1 ELSE 0 END) AS missing_long_description,
    ROUND(SUM(CASE WHEN [Long description] IS NULL OR TRIM([Long description]) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_long_description,

    -- Missing Short Description 2
    SUM(CASE WHEN [Short description 2] IS NULL OR TRIM([Short description 2]) IN ('', 'N/A') THEN 1 ELSE 0 END) AS missing_short_description_2,
    ROUND(SUM(CASE WHEN [Short description 2] IS NULL OR TRIM([Short description 2]) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_short_description_2,

    -- Missing EAN
    SUM(CASE WHEN EAN IS NULL OR TRIM(EAN) IN ('', 'N/A') THEN 1 ELSE 0 END) AS missing_ean,
    ROUND(SUM(CASE WHEN EAN IS NULL OR TRIM(EAN) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_ean,

    -- Missing Technical details
    SUM(CASE WHEN [Technical details] IS NULL OR TRIM([Technical details]) IN ('', 'N/A') THEN 1 ELSE 0 END) AS missing_technical_details,
    ROUND(SUM(CASE WHEN [Technical details] IS NULL OR TRIM([Technical details]) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_technical_details,

    -- Missing Image
    SUM(CASE WHEN Has_image = 0 THEN 1 ELSE 0 END) AS missing_image,
    ROUND(SUM(CASE WHEN Has_image = 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_image

FROM productCatalogDf
GROUP BY Manufacturername
ORDER BY pct_missing_short_description DESC;
"""
pandasql_instance = PandaSQL()
result = pandasql_instance(query, locals())
result

,Manufacturername,total_products,missing_short_description,pct_missing_short_description,missing_long_description,pct_missing_long_description,missing_short_description_2,pct_missing_short_description_2,missing_ean,pct_missing_ean,missing_technical_details,pct_missing_technical_details,missing_image,pct_missing_image
0,BOSCH,140,4,2.9,28,20.0,8,5.7,26,18.6,4,2.9,0,0.0
1,ROTHENBERGER,23,0,0.0,0,0.0,23,100.0,0,0.0,0,0.0,0,0.0
2,GUSTAV KLAUKE GMBH,40,0,0.0,0,0.0,40,100.0,0,0.0,0,0.0,4,10.0
3,FISCHER,5,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,FEIN,100,0,0.0,0,0.0,100,100.0,0,0.0,0,0.0,0,0.0


Assign a simple "data quality score" per product (out of 100)

In [67]:
query = """
SELECT
    Articlenumber,
    Manufacturername,

    -- Count of valid fields
    (
        (CASE WHEN [Short description] IS NOT NULL AND TRIM([Short description]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) +
        (CASE WHEN [Short description 2] IS NOT NULL AND TRIM([Short description 2]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) +
        (CASE WHEN [Long description] IS NOT NULL AND TRIM([Long description]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) +
        (CASE WHEN [Technical details] IS NOT NULL AND TRIM([Technical details]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) +
        (CASE WHEN EAN IS NOT NULL AND TRIM(EAN) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) +
        (CASE WHEN [Picture normal reduced] IS NOT NULL AND TRIM([Picture normal reduced]) <> '' THEN 1 ELSE 0 END) +
        (CASE WHEN [Weight kg] IS NOT NULL AND [Weight kg] > 0 THEN 1 ELSE 0 END) +
        (CASE WHEN [Length m] IS NOT NULL AND [Length m] > 0 THEN 1 ELSE 0 END) +
        (CASE WHEN [Width m] IS NOT NULL AND [Width m] > 0 THEN 1 ELSE 0 END) +
        (CASE WHEN [Depth m] IS NOT NULL AND [Depth m] > 0 THEN 1 ELSE 0 END) +
        (CASE WHEN [ETIM] IS NOT NULL THEN 1 ELSE 0 END) +
        (CASE WHEN [ETIM Features] IS NOT NULL THEN 1 ELSE 0 END) +
        (CASE WHEN [Delivery time days] IS NOT NULL THEN 1 ELSE 0 END) +
        (CASE WHEN Has_image = 1 THEN 1 ELSE 0 END) +
        (CASE WHEN Has_long_description = 1 THEN 1 ELSE 0 END) +
        (CASE WHEN [Description_length] IS NOT NULL AND [Description_length] > 0 THEN 1 ELSE 0 END) +
        (CASE WHEN [Type of product] IS NOT NULL AND TRIM([Type of product]) <> '' THEN 1 ELSE 0 END) +
        (CASE WHEN [Price quantity] IS NOT NULL THEN 1 ELSE 0 END) +
        (CASE WHEN [Volume_m3] IS NOT NULL AND [Volume_m3] > 0 THEN 1 ELSE 0 END)
    ) * (100.0 / 19) AS data_quality_score

FROM productCatalogDf
ORDER BY data_quality_score DESC;
"""
pandasql_instance = PandaSQL()
result = pandasql_instance(query, locals())
result

,Articlenumber,Manufacturername,data_quality_score
0,06016C0000,BOSCH,84.210526
1,06012B4001,BOSCH,84.210526
2,06014A6200,BOSCH,84.210526
3,06014A6000,BOSCH,84.210526
4,06019H6L01,BOSCH,84.210526
...,...,...,...
303,ESG105RMCPCFB,GUSTAV KLAUKE GMBH,57.894737
304,LBOXXEK354CFB,GUSTAV KLAUKE GMBH,57.894737
305,LBOXXEKM60IDCFB,GUSTAV KLAUKE GMBH,57.894737
306,LS100FLEXCFB,GUSTAV KLAUKE GMBH,57.894737


Descriptive Field Interdependency

In [69]:
query = """
SELECT
  CASE
    WHEN Short_description_quality = 'bad'
         AND Short_description_2_quality = 'bad'
         AND Has_long_description = 0 THEN 'Missing All Descriptions'

    WHEN Short_description_quality = 'good'
         AND Short_description_2_quality = 'good'
         AND Has_long_description = 1 THEN 'Complete Descriptions'

    WHEN Short_description_quality = 'bad'
         AND Short_description_2_quality = 'bad'
         AND Has_long_description = 1 THEN 'Only Long Description Present'

    WHEN Short_description_quality = 'bad'
         AND Short_description_2_quality = 'good'
         AND Has_long_description = 0 THEN 'Only Short Description 2 Present'

    WHEN Short_description_quality = 'good'
         AND Short_description_2_quality = 'bad'
         AND Has_long_description = 0 THEN 'Only Short Description 1 Present'

    WHEN Short_description_quality = 'good'
         AND Short_description_2_quality = 'bad'
         AND Has_long_description = 1 THEN 'Short Description 1 + Long Present'

    WHEN Short_description_quality = 'bad'
         AND Short_description_2_quality = 'good'
         AND Has_long_description = 1 THEN 'Short Description 2 + Long Present'

    ELSE 'Other / Mixed'
  END AS description_completeness_combo,
  COUNT(*) AS product_count
FROM productCatalogDf
GROUP BY description_completeness_combo
ORDER BY product_count DESC;
"""
pandasql_instance = PandaSQL()
result = pandasql_instance(query, locals())
result

,description_completeness_combo,product_count
0,Short Description 1 + Long Present,168
1,Complete Descriptions,109
2,Other / Mixed,25
3,Short Description 2 + Long Present,2
4,Only Short Description 1 Present,2
5,Only Short Description 2 Present,1
6,Only Long Description Present,1


In [40]:
query = """
SELECT
    Manufacturername,
    COUNT(*) AS total_products,
    ROUND(AVG(CASE WHEN Short_description_quality = 'good' THEN 1.0 ELSE 0 END) * 100, 2) AS pct_good_short_desc,
    ROUND(AVG(CASE WHEN Short_description_2_quality = 'good' THEN 1.0 ELSE 0 END) * 100, 2) AS pct_good_short_desc_2,
    ROUND(AVG(Has_image) * 100, 2) AS pct_has_image,
    ROUND(AVG(Has_long_description) * 100, 2) AS pct_has_long_desc
FROM productCatalogDf
GROUP BY Manufacturername
ORDER BY pct_good_short_desc + pct_good_short_desc_2 + pct_has_image + pct_has_long_desc ASC;
"""
pandasql_instance = PandaSQL()
result = pandasql_instance(query, locals())
result

,Manufacturername,total_products,pct_good_short_desc,pct_good_short_desc_2,pct_has_image,pct_has_long_desc
0,GUSTAV KLAUKE GMBH,40,100.00,0.00,90.0,100.0
1,FEIN,100,100.00,0.00,100.0,100.0
2,ROTHENBERGER,23,100.00,0.00,100.0,100.0
3,BOSCH,140,97.14,94.29,100.0,80.0
4,FISCHER,5,100.00,100.00,100.0,100.0


In [42]:
query = """
SELECT
    Manufacturername,
    ROUND(AVG(CASE WHEN Short_description_quality = 'good' THEN 1.0 ELSE 0 END) * 100, 2) AS pct_good_short_desc,
    ROUND(AVG(CASE WHEN Short_description_2_quality = 'good' THEN 1.0 ELSE 0 END) * 100, 2) AS pct_good_short_desc_2,
    ROUND(AVG(Has_image) * 100, 2) AS pct_has_image,
    ROUND(AVG(Has_long_description) * 100, 2) AS pct_has_long_desc
FROM productCatalogDf
GROUP BY Manufacturername
ORDER BY Manufacturername;
"""
pandasql_instance = PandaSQL()
result = pandasql_instance(query, locals())
result

,Manufacturername,pct_good_short_desc,pct_good_short_desc_2,pct_has_image,pct_has_long_desc
0,BOSCH,97.14,94.29,100.0,80.0
1,FEIN,100.00,0.00,100.0,100.0
2,FISCHER,100.00,100.00,100.0,100.0
3,GUSTAV KLAUKE GMBH,100.00,0.00,90.0,100.0
4,ROTHENBERGER,100.00,0.00,100.0,100.0


What product variable/column (description or property) usually contains data of good quality per manufacturer? And what is the % of good quality records per variable/column and manufacturer?

In [37]:
query_rel = """
SELECT Manufacturername,
       ((ROUND(100.0 * SUM(CASE WHEN Short_description_quality = 'good' THEN 1 ELSE 0 END) / COUNT(*), 2))+
       (ROUND(100.0 * SUM(CASE WHEN Short_description_2_quality = 'good' THEN 1 ELSE 0 END) / COUNT(*), 2)))/2 as Good_Quality_Product_Percentage
FROM productCatalogDf
GROUP BY Manufacturername
ORDER BY Good_Quality_Product_Percentage DESC;
"""

pandasql_instance = PandaSQL()
result_rel = pandasql_instance(query_rel, locals())
result_rel

,Manufacturername,Good_Quality_Product_Percentage
0,FISCHER,100.000
1,BOSCH,95.715
2,ROTHENBERGER,50.000
3,GUSTAV KLAUKE GMBH,50.000
4,FEIN,50.000
